## Workshop Week 6

## Logistic Regression
Breast Cancer data from [the UCI repository](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains records corresponding to 
cases of observed tumors.   There are a number of observations for each and a categorisation in the `class` column: 2 for benign (good), 4 for malignant (bad).  Your task is to build a logistic regression model to classify these cases. 

The data is provided as a CSV file.  There are a small number of cases where no value is available, these are indicated in the data with `?`. I have used the `na_values` keyword for `read_csv` to have these interpreted as `NaN` (Not a Number).  Your first task is to decide what to do with these rows. You could just drop these rows or you could [impute them from the other data](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).

You then need to follow the procedure outlined in the lecture for generating a train/test set, building and evaluating a model. Your goal is to build the best model possible over this data.   Your first step should be to build a logistic regression model using all of the features that are available.
  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE

In [2]:
bcancer = pd.read_csv('files/breast-cancer-wisconsin.csv', na_values='?')
bcancer.head()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [3]:
# Examine the data: check number of rows and number of columns
bcancer.shape

(699, 11)

In [4]:
# Look at the statistical summary of the dataframe
bcancer.describe()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,683.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.544656,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,3.643857,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [5]:
# Check how many classes we do have from the "class" column
list(set(bcancer['class']))

[2, 4]

In [6]:
# Check number of samples for each class and comment whether dataset is balanced?
print(bcancer[bcancer['class']==2].shape)
print(bcancer[bcancer['class']==4].shape)

(458, 11)
(241, 11)


In [7]:
# Deal with the NaN values in the data
print(bcancer.isnull().sum())
bcancer.dropna(inplace = True)

sample_code_number              0
clump_thickness                 0
uniformity_cell_size            0
uniformity_cell_shape           0
marginal_adhesion               0
single_epithelial_cell_size     0
bare_nuclei                    16
bland_chromatin                 0
normal_nucleoli                 0
mitoses                         0
class                           0
dtype: int64


In [8]:
# Split your data into training(80%) and testing data (20%) and use random_state=142
y = bcancer['class']
X = bcancer.drop(['class', 'sample_code_number'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=142)

In [9]:
# Build your Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [10]:
# Do predictions on test set
y_pred = model.predict(X_test)
pred = pd.concat([pd.Series(y_test.reset_index(drop=True), name='actual'),
           pd.Series(y_pred, name='pred')], axis=1)
pred.head()

,actual,pred
0,2,2
1,2,4
2,2,2
3,2,2
4,4,4


### Evaluation

To evaluate a classification model we want to look at how many cases were correctly classified and how many
were in error.  In this case we have two outcomes - benign and malignant.   SKlearn has some useful tools, the 
[accuracy_score]() function gives a score from 0-1 for the proportion correct.  The 
[confusion_matrix](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function 
shows how many were classified correctly and what errors were made.  Use these to summarise the performance of 
your model (these functions have already been imported above).

In [11]:
# Evaluate the performance of your trained model
accuracy = accuracy_score(y_pred, y_test)
print('Accuracy score: %.2f' % accuracy)

Accuracy score: 0.96


In a binary classification, the **confusion matrix** values:
* $C_{0,0}$ is the number of true negatives (in this case 2 when 2)
* $C_{1,0}$ is the number of false negatives (in this case 4 when 2)
* $C_{1,1}$ is the number of true positives (in this case 4 when 4)
* $C_{0,1}$ is the number of false postivies (in this case 2 when 4)

I think that Python automatically selects the 'negative' as the lowest value which happens to be appropriate for this case.

In [12]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[83  2]
 [ 3 49]]


Now, **sensitivity** and **specificity** are measures of a test's ability to correctly classify an object, in this case the class of breast cancer (2 denoting malignant, and 4 denoting bad).
* The sensitivity refers to a test's ability to designate an individual with disease as positive (bad).
$$Sensitivity = \frac{True\ positives}{True\ positives + False\ negatives}$$
* The specificity of a test is its ability to designate an individual who does not have a disease as negative (malignant).
$$Specificity = \frac{True\ negatives}{True\ negatives + False\ positives}$$

In [13]:
sensitivity = cm[1][1]/(cm[1][1]+cm[1][0])
specificity = cm[0][0]/(cm[0][0]+cm[0][1])
print('Sensitivity: %.2f' % sensitivity)
print('Specificity: %.2f' % specificity)

Sensitivity: 0.94
Specificity: 0.98


### Feature Selection

Since you have many features available, one part of building the best model will be to select which features to use as input to the classifier. Your initial model used all of the features but it is possible that a better model can 
be built by leaving some of them out.   Test this by building a few models with subsets of the features - how do your models perform? 

This process can be automated.  The [sklearn RFE function](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination) implements __Recursive Feature Estimation__ which removes 
features one by one, evaluating the model each time and selecting the best model for a target number of features.  Use RFE to select features for a model with 3, 4 and 5 features - can you build a model that is as good or better than your initial model?

In [17]:
for i in range(3, 6):
    X = bcancer.drop(['class', 'sample_code_number'], axis=1)
    y = bcancer['class']
    
    selector = RFE(model, n_features_to_select=i)
    selector.fit(X, y)
    featured_cols = X.columns[selector.get_support()].tolist()
    
    print(featured_cols)
    
    X_selected = bcancer[featured_cols]
    
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=142)
    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_pred, y_test)
    print('Accuracy score: %.5f' % accuracy)

['uniformity_cell_shape', 'bland_chromatin', 'mitoses']
Accuracy score: 0.92701
['clump_thickness', 'uniformity_cell_shape', 'bland_chromatin', 'mitoses']
Accuracy score: 0.96350
['clump_thickness', 'uniformity_cell_shape', 'bare_nuclei', 'bland_chromatin', 'mitoses']
Accuracy score: 0.96350


Rerunning the logistic regression on `['clump_thickness', 'uniformity_cell_shape', 'bland_chromatin', 'mitoses']`:

In [27]:
X = bcancer.drop(['class', 'sample_code_number', 'bare_nuclei'], axis=1)
y = bcancer['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=142)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_pred, y_test)
print('Accuracy score: %.2f' % accuracy)

cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix:\n', cm)

sensitivity = cm[1][1]/(cm[1][1]+cm[1][0])
specificity = cm[0][0]/(cm[0][0]+cm[0][1])
print('Sensitivity: %.2f' % sensitivity)
print('Specificity: %.2f' % specificity)

Accuracy score: 0.96
Confusion matrix:
 [[83  2]
 [ 3 49]]
Sensitivity: 0.94
Specificity: 0.98


## Analysis
* Based on the RFE, the model containing all variables except `bare_nuclei` achieves the highest accuracy with the least number of variables.
* The model has a very high accuracy score on the test dataset which is desirable and indicates it is a good predictor of class.
* The model has a high sensitivity which means that there are few false negative results, and thus fewer cases of disease are missed.
* The model has a high specificity which means that there are few false positive results.